In [14]:
import pandas as pd
from datetime import datetime
#import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from collections import defaultdict

In [2]:

## Split train and test data set
from sklearn.model_selection import train_test_split
def split_data(X,y):
    # test_size ideal 0.2 ; random_state can be any value
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
    return X_train,X_test,y_train,y_test
############################################ 


In [3]:
def predict_score(X_train,X_test):
    ## Feature scaling
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    ## Fitting model to training set
    classifier_rf = RandomForestClassifier(n_estimators = 10, criterion ='entropy' , random_state = 0)
    classifier_rf.fit(X_train, y_train)
    classifier_lr = LogisticRegression(random_state = 0)
    classifier_lr.fit(X_train, y_train)
    classifier_svc = SVC(kernel = 'rbf', random_state = 0)
    classifier_svc.fit(X_train, y_train)
    classifier_nb = GaussianNB()
    classifier_nb.fit(X_train, y_train)
    ## Predict the result
    y_pred_rf = classifier_rf.predict(X_test)
    score_rf = accuracy_score(y_test, y_pred_rf, normalize=True)
    y_pred_lr = classifier_lr.predict(X_test)
    score_lr = accuracy_score(y_test, y_pred_lr, normalize=True)
    y_pred_svc = classifier_svc.predict(X_test)
    score_svc = accuracy_score(y_test, y_pred_svc, normalize=True)
    y_pred_nb = classifier_nb.predict(X_test)
    score_nb = accuracy_score(y_test, y_pred_nb, normalize=True)

    return y_pred_rf,score_rf,y_pred_lr,score_lr,y_pred_svc,score_svc,y_pred_nb,score_nb,classifier_rf,classifier_lr,classifier_svc,classifier_nb
    
    #return y_pred,score,classifier

In [6]:
match = pd.read_csv('football_results.csv') #whole dataset
data_team = pd.read_csv('team_skills.csv')
country_code = pd.read_csv('country_code.csv') # to fetch 3 letter ISO code
country_code = country_code[['name','alpha-3']]
country_code = dict(zip(country_code['name'],country_code['alpha-3']))
data_team = data_team.replace({"Country": country_code})
match = match.replace({"home_team": country_code})
match = match.replace({"away_team": country_code})
match['match_date'] = pd.to_datetime(match['match_date']) ## Convert datatype to date
match_filtered = match.loc[match['match_date'] > datetime(2010,1,1)] # only matches from 2016
country_array = data_team['Country'].values
match_filtered = match_filtered.loc[(match_filtered['home_team'].isin(country_array)) | (match_filtered['away_team'].isin(country_array))]
match_filtered = match_filtered.reset_index(drop=True)

In [7]:
match_filtered.head()

,match_date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2010-01-02,IRN,PRK,1,0,Friendly,Doha,Qatar,True
1,2010-01-04,EGY,MLI,1,0,Friendly,Dubai,United Arab Emirates,True
2,2010-01-06,KWT,AUS,2,2,AFC Asian Cup qualification,Kuwait City,Kuwait,False
3,2010-01-06,SGP,IRN,1,3,AFC Asian Cup qualification,Singapore,Singapore,False
4,2010-01-06,YEM,JPN,2,3,AFC Asian Cup qualification,Sana'a,Yemen,False


In [6]:
# mlb = MultiLabelBinarizer()

In [7]:
# home_score_list = [[x] for x in match_filtered['home_score'].values]
# away_score_list = [[x] for x in match_filtered['away_score'].values]


In [8]:
# home_score = mlb.fit_transform(home_score_list)

In [9]:
# away_score = mlb.fit_transform(away_score_list)

In [10]:
# np.append(home_score[0],away_score[0])

In [8]:
data_team.head(5)

,Country,Attacking,Midfield,Defence,Overall
0,ARG,87,81,80,82
1,AUS,69,71,73,73
2,BEL,86,83,85,84
3,BRA,86,83,85,85
4,COL,81,77,78,79


In [9]:
match_filtered['winning_team'] = np.where(match_filtered['home_score'] > match_filtered['away_score'],'Home',np.where(match_filtered['home_score'] < match_filtered['away_score'],'Away','Draw'))

## Adding team skills to main data set
match_filtered['home_attack'] = match_filtered.merge(data_team,left_on = 'home_team',right_on ='Country',how = 'left')['Attacking']
match_filtered['home_defense'] = match_filtered.merge(data_team,left_on = 'home_team',right_on ='Country',how = 'left')['Defence']
match_filtered['home_mid'] = match_filtered.merge(data_team,left_on = 'home_team',right_on ='Country',how = 'left')['Midfield']
match_filtered['home_overall'] = match_filtered.merge(data_team,left_on = 'home_team',right_on ='Country',how = 'left')['Overall']

match_filtered['away_attack'] = match_filtered.merge(data_team,left_on = 'away_team',right_on ='Country',how = 'left')['Attacking']
match_filtered['away_defense'] = match_filtered.merge(data_team,left_on = 'away_team',right_on ='Country',how = 'left')['Defence']
match_filtered['away_mid'] = match_filtered.merge(data_team,left_on = 'away_team',right_on ='Country',how = 'left')['Midfield']
match_filtered['away_overall'] = match_filtered.merge(data_team,left_on = 'away_team',right_on ='Country',how = 'left')['Overall']

match_filtered = match_filtered.dropna()
# match_filtered = match_filtered.reset_index()
#match_filtered = match_filtered.reset_index(drop=True)


In [10]:
match_filtered.head()

,match_date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,home_attack,home_defense,home_mid,home_overall,away_attack,away_defense,away_mid,away_overall
7,2010-01-12,EGY,NGA,3,1,African Cup of Nations,Benguela,Angola,True,Home,74.0,71.0,72.0,72.0,77.0,75.0,77.0,77.0
24,2010-01-26,ARG,CRI,3,2,Friendly,San Juan,Argentina,False,Home,87.0,80.0,81.0,82.0,77.0,75.0,77.0,77.0
35,2010-02-14,JPN,KOR,1,3,EAFF Championship,Tokyo,Japan,False,Away,73.0,69.0,74.0,72.0,73.0,69.0,74.0,72.0
41,2010-03-03,BEL,HRV,0,1,Friendly,Brussels,Belgium,False,Away,86.0,85.0,83.0,84.0,77.0,75.0,77.0,77.0
43,2010-03-03,ENG,EGY,3,1,Friendly,London,England,False,Home,84.0,81.0,81.0,82.0,74.0,71.0,72.0,72.0


In [14]:
# match_filtered['att_def']=match_filtered['home_attack']-match_filtered['away_defense']
# match_filtered['mid_mid']=match_filtered['home_mid']-match_filtered['away_mid']
# match_filtered['def_att']=match_filtered['home_defense']-match_filtered['away_attack']
# match_filtered['ovr_ovr']=match_filtered['home_overall']-match_filtered['away_overall']


In [11]:
# Step 1: Importing the dataset
X = match_filtered.iloc[:, 11:19].values # :2 to make it a matrix

y = match_filtered.iloc[:,10].values

X_train,X_test,y_train,y_test = split_data(X,y)

# y_pred_rf,score_rf,y_pred_lr,score_lr,y_pred_svc,score_svc,y_pred_nb,score_nb,classifier_rf,classifier_lr,classifier_svc,classifier_nb = predict_score(X_train,X_test)

In [12]:
match_filtered

,match_date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,home_attack,home_defense,home_mid,home_overall,away_attack,away_defense,away_mid,away_overall
7,2010-01-12,EGY,NGA,3,1,African Cup of Nations,Benguela,Angola,True,Home,74.0,71.0,72.0,72.0,77.0,75.0,77.0,77.0
24,2010-01-26,ARG,CRI,3,2,Friendly,San Juan,Argentina,False,Home,87.0,80.0,81.0,82.0,77.0,75.0,77.0,77.0
35,2010-02-14,JPN,KOR,1,3,EAFF Championship,Tokyo,Japan,False,Away,73.0,69.0,74.0,72.0,73.0,69.0,74.0,72.0
41,2010-03-03,BEL,HRV,0,1,Friendly,Brussels,Belgium,False,Away,86.0,85.0,83.0,84.0,77.0,75.0,77.0,77.0
43,2010-03-03,ENG,EGY,3,1,Friendly,London,England,False,Home,84.0,81.0,81.0,82.0,74.0,71.0,72.0,72.0
44,2010-03-03,FRA,ESP,0,2,Friendly,Saint-Denis,France,False,Away,82.0,81.0,85.0,84.0,84.0,86.0,86.0,86.0
45,2010-03-03,DEU,ARG,0,1,Friendly,Munich,Germany,False,Away,81.0,84.0,85.0,85.0,87.0,80.0,81.0,82.0
55,2010-03-03,CHE,URY,1,3,Friendly,St. Gallen,Switzerland,False,Away,74.0,77.0,78.0,78.0,86.0,79.0,76.0,81.0
60,2010-03-24,MEX,ISL,0,0,Friendly,Charlotte,USA,True,Draw,79.0,76.0,79.0,78.0,73.0,69.0,74.0,72.0
61,2010-04-07,JPN,SRB,0,3,Friendly,Osaka,Japan,False,Away,73.0,69.0,74.0,72.0,73.0,69.0,74.0,72.0


In [18]:
# match_filtered.loc[match_filtered['home_teamam']=='Egypt']

### Obtaining winning streak

In [19]:
dictt_w = defaultdict(int)
dictt_l = defaultdict(int)
dict_h2h = defaultdict(int)
home_win_streak = []
away_win_streak = []
home_lose_streak = []
away_lose_streak = []
for i, row in match_filtered.iterrows():
    home_win_streak.append(dictt_w[row['home_team']])
    away_win_streak.append(dictt_w[row['away_team']])
    home_lose_streak.append(dictt_l[row['home_team']])
    away_lose_streak.append(dictt_l[row['away_team']])
    if row['winning_team'] == 'Home':
        dictt_w[row['home_team']] += 1
        dictt_w[row['away_team']] = 0
        dictt_l[row['away_team']] += 1
        dictt_l[row['home_team']] = 0
    elif row['winning_team'] == 'Away':
        dictt_w[row['away_team']] += 1
        dictt_w[row['home_team']] = 0
        dictt_l[row['home_team']] += 1
        dictt_l[row['away_team']] = 0
match_filtered['home_win_steak']= home_win_streak
match_filtered['away_win_steak']= away_win_streak
match_filtered['home_lose_steak']= home_lose_streak
match_filtered['away_lose_steak']= away_lose_streak

In [16]:
m1 = match_filtered.loc[:,['match_date','home_team','away_team', 'home_score' , 'away_score', 'winning_team', 'home_win_streak', 'away_win_streak','home_lose_streak','away_lose_streak']]

m2 = m1.loc[((match_filtered['home_team'] == 'ENG') | (match_filtered['away_team'] == 'ENG')) & ((match_filtered['home_team'] == 'DEU') | (match_filtered['away_team'] == 'DEU'))]

/home/ebby/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [18]:
m2

,match_date,home_team,away_team,home_score,away_score,winning_team,home_win_streak,away_win_streak,home_lose_streak,away_lose_streak
155,2010-06-27,DEU,ENG,4,1,Home,NaN,NaN,NaN,NaN
1361,2013-11-19,ENG,DEU,0,1,Away,NaN,NaN,NaN,NaN
2028,2016-03-26,DEU,ENG,2,3,Away,NaN,NaN,NaN,NaN
2355,2017-03-22,DEU,ENG,1,0,Home,NaN,NaN,NaN,NaN
2599,2017-11-10,ENG,DEU,0,0,Draw,NaN,NaN,NaN,NaN


### Import WC fixture


In [49]:

data_fixture = pd.read_csv('fifa_2018.csv')

## Adding team skills to main data set
data_fixture['home_attack'] = data_fixture.merge(data_team,left_on = 'Home Team',right_on ='Country',how = 'left')['Attacking']
data_fixture['home_defense'] = data_fixture.merge(data_team,left_on = 'Home Team',right_on ='Country',how = 'left')['Defence']
data_fixture['home_mid'] = data_fixture.merge(data_team,left_on = 'Home Team',right_on ='Country',how = 'left')['Midfield']
data_fixture['home_overall'] = data_fixture.merge(data_team,left_on = 'Home Team',right_on ='Country',how = 'left')['Overall']

data_fixture['away_attack'] = data_fixture.merge(data_team,left_on = 'Away Team',right_on ='Country',how = 'left')['Attacking']
data_fixture['away_defense'] = data_fixture.merge(data_team,left_on = 'Away Team',right_on ='Country',how = 'left')['Defence']
data_fixture['away_mid'] = data_fixture.merge(data_team,left_on = 'Away Team',right_on ='Country',how = 'left')['Midfield']
data_fixture['away_overall'] = data_fixture.merge(data_team,left_on = 'Away Team',right_on ='Country',how = 'left')['Overall']
# X = data_fixture.iloc[:, 6:14]
# X = X.dropna()
# X = X.values



In [39]:
data_fixture

,Round Number,Date,Home Team,Away Team,Group,Result,home_attack,home_defense,home_mid,home_overall,away_attack,away_defense,away_mid,away_overall
0,1,14/06/2018 18:00,Russia,Saudi Arabia,Group A,NaN,80.0,76.0,78.0,78.0,73.0,73.0,73.0,73.0
1,1,15/06/2018 15:00,Egypt,Uruguay,Group A,NaN,74.0,71.0,72.0,72.0,86.0,79.0,76.0,81.0
2,1,15/06/2018 18:00,Morocco,Iran,Group B,NaN,73.0,69.0,74.0,72.0,73.0,69.0,74.0,72.0
3,1,15/06/2018 21:00,Portugal,Spain,Group B,NaN,85.0,80.0,82.0,82.0,84.0,86.0,86.0,86.0
4,1,16/06/2018 13:00,France,Australia,Group C,NaN,82.0,81.0,85.0,84.0,69.0,73.0,71.0,73.0
5,1,16/06/2018 16:00,Argentina,Iceland,Group D,NaN,87.0,80.0,81.0,82.0,73.0,69.0,74.0,72.0
6,1,16/06/2018 19:00,Peru,Denmark,Group C,NaN,73.0,69.0,74.0,72.0,77.0,75.0,77.0,77.0
7,1,16/06/2018 22:00,Croatia,Nigeria,Group D,NaN,77.0,75.0,77.0,77.0,77.0,75.0,77.0,77.0
8,1,17/06/2018 15:00,Costa Rica,Serbia,Group E,NaN,77.0,75.0,77.0,77.0,73.0,69.0,74.0,72.0
9,1,17/06/2018 18:00,Germany,Mexico,Group F,NaN,81.0,84.0,85.0,85.0,79.0,76.0,79.0,78.0


In [56]:
match_filtered['list_score'] = np.array(zip(match_filtered['home_score'], match_filtered['away_score']))

In [22]:
y1 = match_filtered['home_score'].values

In [23]:
y2 = match_filtered['away_score'].values

In [24]:
X.shape

(557, 4)

In [25]:
logModel1 = LogisticRegression()
logModel2 = LogisticRegression()

In [26]:
X_train,X_test,y1_train,y1_test = split_data(X,y1)

In [27]:

X_train,X_test,y2_train,y2_test = split_data(X,y2)

In [28]:
y1_train.shape

(417,)

In [30]:
y1_model = logModel1.fit(X_train, y1_train)

In [31]:
y2_model = logModel2.fit(X_train, y2_train)

In [32]:
y1_pred = y1_model.predict(X_test)
y2_pred = y2_model.predict(X_test)

score1 = accuracy_score(y1_test, y1_pred, normalize=True)
score2 = accuracy_score(y2_test, y2_pred, normalize=True)

In [33]:
score1

0.2857142857142857

In [34]:
score2

0.37142857142857144

In [ ]:
85.0, 80.0, 82.0, 82.0, 84.0, 86.0, 86.0, 86.0

In [42]:
saud_rus = np.array([[4,-6, -3, 5]])

In [43]:
y1_pred = y1_model.predict(saud_rus)
y2_pred = y2_model.predict(saud_rus)

In [44]:
y2_pred

array([1])

In [45]:
y1_pred

array([3])

In [102]:
score

0.3357142857142857

In [74]:
# y = np.asarray(list(zip(match_filtered['home_score'], match_filtered['away_score'])))

In [75]:
type(y)

numpy.ndarray

In [68]:
y[1]

array([3, 2])

In [64]:
y.shape

(557, 2)

In [54]:
y = match_filtered.iloc[:,-1].values

In [55]:
y

array([list([3, 1]), list([3, 2]), list([1, 3]), list([0, 1]),
       list([3, 1]), list([0, 2]), list([0, 1]), list([1, 3]),
       list([0, 0]), list([0, 3]), list([1, 0]), list([3, 1]),
       list([0, 2]), list([0, 0]), list([2, 1]), list([2, 0]),
       list([3, 2]), list([1, 2]), list([1, 1]), list([1, 1]),
       list([1, 0]), list([0, 1]), list([0, 0]), list([1, 0]),
       list([6, 0]), list([0, 0]), list([1, 0]), list([4, 0]),
       list([0, 1]), list([4, 1]), list([0, 2]), list([0, 1]),
       list([0, 1]), list([2, 2]), list([2, 1]), list([1, 3]),
       list([0, 0]), list([2, 1]), list([3, 1]), list([4, 1]),
       list([1, 0]), list([0, 4]), list([0, 1]), list([2, 3]),
       list([2, 2]), list([2, 1]), list([1, 1]), list([0, 1]),
       list([2, 2]), list([0, 0]), list([4, 1]), list([1, 0]),
       list([0, 1]), list([1, 0]), list([1, 2]), list([1, 3]),
       list([0, 3]), list([1, 0]), list([2, 0]), list([3, 1]),
       list([1, 3]), list([0, 0]), list([1, 0]), list([

In [53]:
X

array([[74., 71., 72., ..., 75., 77., 77.],
       [87., 80., 81., ..., 75., 77., 77.],
       [73., 69., 74., ..., 69., 74., 72.],
       ...,
       [73., 73., 73., ..., 69., 74., 72.],
       [84., 86., 86., ..., 77., 78., 78.],
       [77., 75., 77., ..., 85., 83., 85.]])

In [15]:
match_filtered.count()

date          2734
home_team     2734
away_team     2734
home_score    2734
away_score    2734
tournament    2734
city          2734
country       2734
neutral       2734
list_score    2734
dtype: int64